Remember to turn on GPU accelerator on the right tab

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd
import random

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 888

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

In [ ]:
directory_train = "../input/cv-assignment-2-224x224-training-set"

pixels = 224
batch_size = 64
num_class = 75

def generate_data_df_with_folds(kfold=10):
    files = pd.DataFrame()

    tmp = []
    labels_1 = []
    for i in range(75):
        for x,_,z in os.walk(directory_train+'/'+str(i)):
            for file in z:
                if 'desktop.ini' not in file:
                    filepath = x+'/'+file
                    tmp.append(filepath)
                    labels_1.append(i)

    labels_2 = [1 if i in range(15,74) else 0 for i in labels_1] #1 if food, 0 if place
    files['filepaths'] = pd.Series(tmp)
    files['target'] = pd.Series(labels_1)

    folds = []
    for i in range(75):
        n = files[files['target']==i].shape[0]
        tmp = []
        for fold in range(kfold):
            if fold != kfold-1:
                tmp += [fold]*(n//kfold)
            else:
                tmp+= [fold]*(n-len(tmp))
        random.shuffle(tmp)
        folds+=tmp
    files['fold'] = folds
    files['target'] = files['target'].astype(str)
    return files

files = generate_data_df_with_folds(kfold=5)

In [ ]:
gen = tf.keras.preprocessing.image.ImageDataGenerator()
train = gen.flow_from_dataframe(files[files['fold']>1],x_col='filepaths',y_col='target',
                                class_mode='sparse',batch_size=batch_size,target_size=(224,224))
val = gen.flow_from_dataframe(files[files['fold']<=1],x_col='filepaths',y_col='target',
                                class_mode='sparse',batch_size=batch_size,target_size=(224,224),shuffle=False)

In [ ]:
### modelling stuff

In [ ]:
def create_mapping():
    import pandas as pd
    mapper = pd.DataFrame()
    mapper['range']=[str(x) for x in range(75)]
    mapper.sort_values(by='range',inplace=True)
    mapper.reset_index(drop=True,inplace=True)
    mapper.reset_index(inplace=True)
    mapping_dict = {k:int(v) for k,v in zip(mapper['index'],mapper['range'])}
    return mapping_dict

mapping_dict = create_mapping()

In [ ]:
pred_val = model.predict(val)

In [ ]:
val_df = pd.DataFrame()
val_df['paths'] = files[files['fold']==0]['filepaths']
val_df['pred'] = pred_val.argmax(axis=-1)
val_df['pred'] = [mapping_dict[x] for x in val_df['pred']]
val_df['true']=[int(x.replace(directory_train+'/','').replace('.jpg','').split('/')[0]) for x in val_df['paths']]
val_df['id']=[int(x.replace(directory_train+'/','').replace('.jpg','').split('/')[1]) for x in val_df['paths']]

from sklearn.metrics import classification_report

print(classification_report(val_df['true'],val_df['pred']))

In [ ]:
test = tf.keras.utils.image_dataset_from_directory('../input/food-places-test-224x224/processed_test',
                                                   labels=None,image_size=(224,224),shuffle=False)
pred_test = model.predict(test)

In [ ]:
test_df = pd.DataFrame()
test_df['Id'] = [int(x.replace('../input/food-places-test-224x224/processed_test/','').replace('.jpg','')) for x in test.file_paths]
top_5 = pred_test.argsort(axis=-1)[:,-5:]
test_df[[5-i for i in range(5)]] = top_5
for i in range(1,6):
    test_df[i] = [mapping_dict[x] for x in test_df[i]]
test_df.sort_values(by='Id',inplace=True)
test_df.reset_index(drop=True,inplace=True)
test_df = test_df[['Id',1,2,3,4,5]]
test_df.columns = ['Id','Top 1','Top 2','Top 3','Top 4','Top 5']

In [ ]:
test_df[['Id','Top 1']].to_csv('submission.csv',index=False)